<a href="https://colab.research.google.com/github/mcadete/LLM_4_Biz_16/blob/main/tech16_class3_inclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index

In [2]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)


In [3]:
%pip install pypdf

In [4]:
!wget https://privatewealth.goldmansachs.com/outlook/2025-isg-outlook.pdf

--2025-02-18 02:56:03--  https://privatewealth.goldmansachs.com/outlook/2025-isg-outlook.pdf
Resolving privatewealth.goldmansachs.com (privatewealth.goldmansachs.com)... 23.53.13.170, 23.53.13.171
Connecting to privatewealth.goldmansachs.com (privatewealth.goldmansachs.com)|23.53.13.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘2025-isg-outlook.pdf.1’

2025-isg-outlook.pd     [ <=>                ]   6.88M  35.1MB/s    in 0.2s    

2025-02-18 02:56:03 (35.1 MB/s) - ‘2025-isg-outlook.pdf.1’ saved [7209986]



In [5]:
import os
os.environ["OPENAI_API_KEY"] = open_ai_key

In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [7]:
documents = SimpleDirectoryReader('./').load_data('2025-isg-outlook.pdf')

Loading files: 100%|██████████| 2/2 [00:15<00:00,  7.77s/file]


In [8]:
documents[10]

Document(id_='23399128-25eb-46da-9690-2195d6f777d2', embedding=None, metadata={'page_label': '9', 'file_name': '2025-isg-outlook.pdf', 'file_path': '/content/2025-isg-outlook.pdf', 'file_type': 'application/pdf', 'file_size': 7209986, 'creation_date': '2025-02-18', 'last_modified_date': '2025-02-17'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='9Outlook Investment Strategy Group\n SECTION II: DIVERGING PATHS\n58  2025 Global  \nEconomic Outlook\n  Despite divergent paths, most economies will \nascend toward higher elevations this year.  \n60  United States\n64  Eurozone\n66  United Kingdom\n67 Japan\n68  Emerging Markets\n SECTION III:

In [9]:
index = VectorStoreIndex.from_documents(documents)

In [10]:
query_engine = index.as_query_engine()

In [11]:
response = query_engine.query("What is the outlook for developing markets?")

In [12]:
print(response)

The outlook for developing markets suggests that several Latin American currencies faced challenges despite decent economic growth and supportive monetary policy. The Brazilian real and Mexican peso were particularly affected by policy uncertainty. Despite concerns about US dollar strength potentially weakening, the expectation for 2025 is another year of gains. Higher US interest rates compared to other developed markets are likely to attract capital inflows, with US growth expected to outpace offshore peers. These factors support a forecast for low-single-digit US dollar returns.


# Adding memory

In [13]:
response = query_engine.query("What did I just ask you?. Give me the exact question ")
print(response)

What did I just ask you?. Give me the exact question.


In [14]:
from re import VERBOSE
chat_engine = index.as_chat_engine(chat_mode="openai", verbose=True)
response = chat_engine.chat("Hello there")
print(response)

Added user message to memory: Hello there
Hi! How can I assist you today?


In [15]:
response = chat_engine.chat("What is the liklihood of a recession in 2025?")
print(response)

Added user message to memory: What is the liklihood of a recession in 2025?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"What is the likelihood of a recession in 2025?"}
Got output: The likelihood of a recession in 2025 is 20%, which is close to the long-term average but below the current consensus.

The likelihood of a recession in 2025 is estimated to be around 20%, which is close to the long-term average but below the current consensus.


In [16]:
response = chat_engine.chat("What did I just ask you?")

Added user message to memory: What did I just ask you?


In [17]:
print(response)

You asked about the likelihood of a recession in 2025.


# Changing settings - chunking

In [18]:
from llama_index.core import Settings

Settings.chunk_size = 10000
Settings.chunk_overlap = 200

# Creating VectorDB

In [19]:
%pip install llama-index-vector-stores-chroma

In [20]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [21]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [25]:
chroma_collection = chroma_client.delete_collection("tech16")
chroma_collection = chroma_client.create_collection("tech16")

In [27]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [28]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./").load_data("2025-isg-outlook.pdf")
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
query_engine = index.as_query_engine()
response = query_engine.query("What is the 2025 outlook for US GDP?")
print(response)

Loading files: 100%|██████████| 2/2 [00:12<00:00,  6.00s/file]


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 9410 tokens (9410 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [29]:
!pip install youtube-transcript-api

In [30]:
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()
documents = loader.load_data(
    ytlinks=["https://www.youtube.com/watch?v=dQw4w9WgXcQ"]
)

ModuleNotFoundError: No module named 'llama_index.readers.youtube_transcript'

In [31]:
pip install llama-index-readers-slack

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.5/292.5 kB 5.0 MB/s eta 0:00:00


In [ ]:
from llama_index.readers.slack import SlackReader

reader = SlackReader(
    slack_token = ""
)

In [32]:
pip install llama-index openai chromadb beautifulsoup4 requests markdownify llama-hub fastapi uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 MB 5.5 MB/s eta 0:00:00
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=48f341e2d37514d1fbaf37378d20610ae37f4625796033e045df42826bf1bad3
  Stored in directory: /root/.cache/pip/wheels/23/58/7c/d9c8c4d924a1ac2b621add1b2c1d30b639629a33cfdfde6a45
Successfully built html2text


In [33]:
import os
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md
from llama_index import SimpleDirectoryReader, VectorStoreIndex, ServiceContext
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.llms import OpenAI
from fastapi import FastAPI, HTTPException
import uvicorn

# Set OpenAI API Key (replace with your own key)
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection("k8s_docs")

# Function to scrape GitHub markdown content
def scrape_github_markdown(repo_url):
    raw_url = repo_url.replace("github.com", "raw.githubusercontent.com").replace("/tree/main", "")
    response = requests.get(raw_url)
    if response.status_code == 200:
        return response.text
    return ""

# Function to scrape Kubernetes docs
def scrape_kubernetes_docs(base_url):
    response = requests.get(base_url)
    if response.status_code != 200:
        return ""

    soup = BeautifulSoup(response.text, "html.parser")
    text_content = soup.get_text(separator=" ", strip=True)
    return text_content

# Scrape and store data
print("Scraping Kubernetes documentation...")
github_data = scrape_github_markdown("https://github.com/kelseyhightower/kubernetes-the-hard-way/blob/main/README.md")
kubernetes_docs_data = scrape_kubernetes_docs("https://kubernetes.io/docs/")

# Save scraped content to files
os.makedirs("data", exist_ok=True)
with open("data/kubernetes_the_hard_way.md", "w", encoding="utf-8") as f:
    f.write(md(github_data))

with open("data/kubernetes_docs.txt", "w", encoding="utf-8") as f:
    f.write(kubernetes_docs_data)

# Load documents into LlamaIndex
reader = SimpleDirectoryReader(input_dir="data")
documents = reader.load_data()

# Create vector store index with ChromaDB
service_context = ServiceContext.from_defaults(embed_model=OpenAIEmbedding())
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context)

# Create a FastAPI app for the chat interface
app = FastAPI()

@app.get("/ask")
def ask_kubernetes_question(query: str):
    try:
        query_engine = index.as_query_engine()
        response = query_engine.query(query)
        return {"response": str(response)}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    print("Starting Kubernetes Chatbot API on http://127.0.0.1:8000/ask?query=your-question")
    uvicorn.run(app, host="0.0.0.0", port=8000)


ImportError: cannot import name 'SimpleDirectoryReader' from 'llama_index' (unknown location)